In [15]:
import tkinter as tk
import winsound
import time


class Alien(object):
    image_alien = None
    image_width = None
    image_height = None
    
    def __init__(self):
        self.id = None
        self.alive = True
    
    @classmethod
    def get_image(cls):
        if cls.image_alien == None:
            cls.image_alien = tk.PhotoImage(file='alien.gif')
            cls.image_width = tk.PhotoImage.width(cls.image_alien)
            cls.image_height = tk.PhotoImage.height(cls.image_alien)
        return cls.image_alien
    
    def install_in(self, canvas, x, y):
        alien = Alien.get_image()
        self.id = canvas.create_image(x, y, image=alien)
        
    def touched_by(self, canvas, bullet):
        self.alive = False
        canvas.delete(self.id)
        canvas.move(bullet.id, 0, - 700)
    
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id, dx, dy)
        
class Fleet(object):
    image_explosion = None
    
    @classmethod
    def get_image(cls):
        if cls.image_explosion == None:
            cls.image_explosion = tk.PhotoImage(file='explosion.gif')
        return cls.image_explosion
    
    def __init__(self):
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 20
        self.alien_x_delta = 5
        self.alien_y_delta = 15
        self.alien_width = None
        self.alien_id = []
        self.explo = 0
        self.location_explo = None
        self.score = 0
        self.affichage_score = None
        
    def install_in(self, canvas):
        alien = Alien()
        Alien.get_image()
        y= -self.alien_x_delta
        for i in range (0, self.aliens_lines):
            y = y + Alien.image_height + self.alien_y_delta
            x = 0
            for j in range (0, self.aliens_columns):
                x = x + Alien.image_width + self.alien_x_delta
                alien = Alien()
                self.alien_width = alien.image_width
                alien.install_in(canvas, x, y)  
                self.alien_id.append(alien)
                
    def explosionend(self,canvas,explo):
        canvas.delete(explo)
                
    def explosion(self,canvas,x, y):
        winsound.PlaySound("explos.wav", winsound.SND_ASYNC)
        image_explosion = Fleet.get_image()
        explo = canvas.create_image(x, y, image=image_explosion)
        self.explo=1
        self.location_explo = explo
    
    def manage_touched_aliens_by(self,canvas,bullet):
        y= -self.alien_x_delta
        x1, y1, x2, y2 = canvas.coords(bullet.id)
        target = canvas.find_overlapping(x1, y1, x2, y2)
        if (len(target)==2):
            for alien in self.alien_id:
                xa, ya = canvas.coords(alien.id)
                if (xa<=x1<=xa+73 and ya<=y1<=ya+53):
                    alien.touched_by(canvas, bullet)
                    self.alien_id.remove(alien)
                    self.explosion(canvas, x1, y1)
                    canvas.delete(self.affichage_score)
                    self.score = self.score + 1
                    affiche_s = "score : " + str(self.score)
                    self.affichage_score = canvas.create_text(845, 15,font=("Purisa", 18), text=affiche_s, fill='white')
                    if len(self.alien_id)==0:
                        canvas.create_text(450, 350,font=("Purisa", 18), text='Fin du jeu vous avez gagné !', fill='white')
        
    
class Defender(object):
    def __init__(self):
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets = 8
        self.fired_bullets = []
        
    def install_in(self, canvas):
        x = int(canvas['width'])//2
        y = int(canvas['height'])- self.height
        self.id = canvas.create_rectangle(x, y, x+self.width, y +self.height , fill='white')
    
    def move_in(self,canvas, dx):
        x = int(canvas.coords(self.id)[0])
        if x+dx < 0:
            dx=0
        elif x+dx > int(canvas['width'])-self.width:
            dx=0
        else:
            canvas.move(self.id, dx, 0)
    
    def fire(self, canvas):
        if (len(self.fired_bullets) < self.max_fired_bullets):
            newBullet = Bullet(self)
            newBullet.install_in(canvas)
            self.fired_bullets.append(newBullet)
    
        
class Bullet(object):
    def __init__(self, shooter):
        self.radius = 8
        self.color = "red"
        self.speed = 20
        self.id = None
        self.shooter = shooter
        
    def install_in(self, canvas):
        x1, y1, x2, y2 = canvas.coords(self.shooter.id)
        x = x1 + self.radius
        y = y1 - 3*self.radius
        self.id = canvas.create_oval(x, y, x + self.radius, y + self.radius, fill='red')
    
    def move_in(self, canvas):
        canvas.move(self.id, 0, - self.speed)
        

class Game (object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.width=900
        self.height =800
        self.canvas = tk.Canvas(self.frame, width=self.width, height=self.height, bg='black')
        self.canvas.pack()
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
        self.fleet_position_x1 = 0
        self.fleet_position_x2 = (self.fleet.alien_width + self.fleet.alien_x_delta) * self.fleet.aliens_columns
        self.dx= 20
        self.dy = 0
        self.fin = 0
        
    def keypress(self, event):
        dx, dy = 0, 0
        if event.keysym == 'Left' or event.keysym == 'q':
            self.defender.move_in(self.canvas, -10)
        elif event.keysym == 'Right' or event.keysym == 'd':
            self.defender.move_in(self.canvas, 10)
        elif event.keysym == 'space':
            self.defender.fire(self.canvas)
            winsound.PlaySound("pistolet-laser.wav", winsound.SND_ASYNC)
                    
            
    def animation(self):
        if (self.fleet.explo==1):
            self.fleet.explosionend(self.canvas, self.fleet.location_explo)
            self.fleet.explo = 0
        if (self.fin==0):
            self.move_bullets()
            self.move_aliens_fleet()
            self.canvas.after(300, self.animation)
    
    def start_animation(self):
        self.canvas.after(10, self.animation)    
        
    def move_bullets(self):
        for bullet in self.defender.fired_bullets:
            x1, y1, x2, y2 =self.canvas.coords(bullet.id)
            self.fleet.manage_touched_aliens_by(self.canvas, bullet)
            bullet.move_in(self.canvas)
            if y1 < 0:
                self.canvas.delete(bullet.id)
                self.defender.fired_bullets.remove(bullet)
    
    def move_aliens_fleet(self):
        for fleet in self.fleet.alien_id:
            fleet.move_in(self.canvas, self.dx, self.dy)
        self.fleet_position_x1= self.fleet_position_x1 + self.dx
        self.fleet_position_x2= self.fleet_position_x2 + self.dx
        if(self.fleet_position_x1<=(0 - self.fleet.aliens_columns - (2*self.fleet.alien_x_delta) - 1) or self.fleet_position_x2>= (self.width - self.fleet.aliens_columns*self.fleet.alien_x_delta)):
                self.dx = -self.dx
                self.dy = self.dy+ 0.05
                target = self.canvas.find_overlapping(0, (self.height - self.defender.height - 20), self.width, self.height)
                if (len(target)>=2):
                    self.canvas.create_text(450, 350,font=("Purisa", 18), text='Fin du jeu vous avez perdu !', fill='white')
                    self.fin = 1

        
class SpaceInvaders(object):
    
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both")
        self.game = Game(self.frame)

    def play(self):
        self.game.start_animation()
        self.root.bind("<Key>", self.game.keypress)
        self.root.mainloop()
                        
                        
SpaceInvaders().play()